## This script accesses the CESM2 Precipitation data for the Scenario years from GLADE repository of NCAR

In [1]:
import xarray as xr
from netCDF4 import Dataset 
import numpy as np
from numpy import unravel_index

from matplotlib import pyplot as plt 
from numpy import linspace

In [2]:
# Projection
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
from descartes import PolygonPatch
import shapefile as shp  # Requires the pyshp package
import cartopy.io.shapereader as shpreader

In [4]:
# Common imports
import calendar
import datetime
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from numpy import array, ma
import scipy.io

In [6]:
cd '/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PRECT'

/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PRECT


In [7]:
pwd

'/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PRECT'

## Accessing One File

In [8]:
filePath='/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PRECT'
fileName_Sample = 'b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.PRECT.185001-185912.nc'
ncFile_precip_Sample = str(filePath) + '/' + str(fileName_Sample)
print(ncFile_precip_Sample)

/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PRECT/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.PRECT.185001-185912.nc


In [9]:
file_xr_Sample  = xr.open_dataset(ncFile_precip_Sample) 
file_xr_Sample

<xarray.Dataset>
Dimensions:       (ilev: 33, lat: 192, lev: 32, lon: 288, nbnd: 2, time: 120, zlon: 1)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * zlon          (zlon) float64 0.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) datetime64[ns] 1850-02-01 1850-03-01 ... 1860-01-01
Dimensions without coordinates: nbnd
Data variables:
    zlon_bnds     (zlon, nbnd) float64 ...
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    hybi          (ilev) float64 ...
    ndbase        float64 ...
    nsbase        float64 ...
    nbdate        float64 ...
    nbsec         float64 ...
    mdt           float64 

In [10]:
file_precip_Sample = Dataset(ncFile_precip_Sample, mode='r')
file_precip_Sample

lons = file_precip_Sample.variables['lon'][:]
lats = file_precip_Sample.variables['lat'][:]
times = file_precip_Sample.variables['time'][:]
ntim = times.shape[0]
print(ntim)

time = np.arange(ntim)
time

param_precip_Sample = file_precip_Sample.variables['PRECT'][:]*86400*30*1000
print(param_precip_Sample.shape)
print('')

120
(120, 192, 288)



In [11]:
t,y,x = param_precip_Sample.shape
print(y)
print(x)
print(t)

#nDecades = 16
nDecades = 8

monStepHistTot = t*nDecades + 72   # =185001_201412 (60 is half of 120)
print(monStepHistTot)

ensMem = 50
print(ensMem)

192
288
120
1032
50


In [12]:
print(lats)

[-90.         -89.05759162 -88.11518325 -87.17277487 -86.23036649
 -85.28795812 -84.34554974 -83.40314136 -82.46073298 -81.51832461
 -80.57591623 -79.63350785 -78.69109948 -77.7486911  -76.80628272
 -75.86387435 -74.92146597 -73.97905759 -73.03664921 -72.09424084
 -71.15183246 -70.20942408 -69.26701571 -68.32460733 -67.38219895
 -66.43979058 -65.4973822  -64.55497382 -63.61256545 -62.67015707
 -61.72774869 -60.78534031 -59.84293194 -58.90052356 -57.95811518
 -57.01570681 -56.07329843 -55.13089005 -54.18848168 -53.2460733
 -52.30366492 -51.36125654 -50.41884817 -49.47643979 -48.53403141
 -47.59162304 -46.64921466 -45.70680628 -44.76439791 -43.82198953
 -42.87958115 -41.93717277 -40.9947644  -40.05235602 -39.10994764
 -38.16753927 -37.22513089 -36.28272251 -35.34031414 -34.39790576
 -33.45549738 -32.51308901 -31.57068063 -30.62827225 -29.68586387
 -28.7434555  -27.80104712 -26.85863874 -25.91623037 -24.97382199
 -24.03141361 -23.08900524 -22.14659686 -21.20418848 -20.2617801
 -19.3193717

In [13]:
print(lons)

[  0.     1.25   2.5    3.75   5.     6.25   7.5    8.75  10.    11.25
  12.5   13.75  15.    16.25  17.5   18.75  20.    21.25  22.5   23.75
  25.    26.25  27.5   28.75  30.    31.25  32.5   33.75  35.    36.25
  37.5   38.75  40.    41.25  42.5   43.75  45.    46.25  47.5   48.75
  50.    51.25  52.5   53.75  55.    56.25  57.5   58.75  60.    61.25
  62.5   63.75  65.    66.25  67.5   68.75  70.    71.25  72.5   73.75
  75.    76.25  77.5   78.75  80.    81.25  82.5   83.75  85.    86.25
  87.5   88.75  90.    91.25  92.5   93.75  95.    96.25  97.5   98.75
 100.   101.25 102.5  103.75 105.   106.25 107.5  108.75 110.   111.25
 112.5  113.75 115.   116.25 117.5  118.75 120.   121.25 122.5  123.75
 125.   126.25 127.5  128.75 130.   131.25 132.5  133.75 135.   136.25
 137.5  138.75 140.   141.25 142.5  143.75 145.   146.25 147.5  148.75
 150.   151.25 152.5  153.75 155.   156.25 157.5  158.75 160.   161.25
 162.5  163.75 165.   166.25 167.5  168.75 170.   171.25 172.5  173.75
 175. 

## Setting US boundaries

In [14]:
# Save only U.S. 
latboundsUs = [ 25 , 55]
lonboundsUs = [230 , 300]

In [15]:
# latitude lower and upper index
latli = np.argmin( np.abs( lats - latboundsUs[0] ) )
latui = np.argmin( np.abs( lats - latboundsUs[1] ) ) 

# longitude lower and upper index
lonli = np.argmin( np.abs( lons - lonboundsUs[0] ) )
lonui = np.argmin( np.abs( lons - lonboundsUs[1] ) )  

In [16]:
print(latli, latui)

122 154


In [17]:
print(lonli, lonui)

184 240


In [18]:
print(param_precip_Sample.shape)

(120, 192, 288)


In [19]:
param_precip_Sample_US = param_precip_Sample[:, latli:latui , lonli:lonui]
print(param_precip_Sample_US.shape)

(120, 32, 56)


In [20]:
t,y_US,x_US = param_precip_Sample_US.shape
print(y_US)
print(x_US)
print(t)

32
56
120


In [22]:
#
monPrecipUsScenEnsMem = np.zeros((monStepHistTot, y_US, x_US, ensMem, )) 
monPrecipUsScenEnsMem[:] = np.nan
print(monPrecipUsScenEnsMem.shape)

(1032, 32, 56, 50)


In [23]:

filePath = '/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/month_1/PRECT'

yrCtrlIds = ['1001', '1021', '1041', '1061', '1081', '1101', '1121', '1141', '1161', '1181', '1231', '1251', '1281', '1301'];
yrId = 0
ctrlRunId = 0

monSt = 1
monEnd = 12

memId = -1
for yrCtrl in yrCtrlIds:    
    yrId = yrId + 1   
    
    if yrId < 11:
        memId = memId + 1
        tStepSt = 0
        
        yrHistSt = 2014        
        yrHistEnd = yrHistSt + 9
    
        ctrlRunId = ctrlRunId + 1  
        
        yrCtrlDigits = str(yrCtrl)+'.' + str(ctrlRunId).rjust(3,'0')        
        print(yrCtrlDigits)
        
        for decade in range(9):
            
            yrHistSt = yrHistSt + 1
            yrHistEnd = yrHistEnd + 1
            
            yrMonHistSt = str(yrHistSt) + str(monSt).rjust(2,'0')
            yrMonHistEnd = str(yrHistEnd) + str(monEnd).rjust(2,'0')              
                        
            
            fileName = 'b.e21.BSSP370cmip6.f09_g17.LE2-'+yrCtrlDigits+'.cam.h0.PRECT.' +yrMonHistSt+'-'+yrMonHistEnd+'.nc'
            
            ncFile_precip = str(filePath) + '/' + str(fileName)
            file_precip = Dataset(ncFile_precip, mode='r')                        
            param_precipToAdd = file_precip.variables['PRECT'][:, latli:latui , lonli:lonui]*86400*30*1000            
            t,y_US,x_US = param_precipToAdd.shape
            tStepEnd = tStepSt + t            
            
                        
            monPrecipUsScenEnsMem[tStepSt:tStepEnd, :, :, memId] = param_precipToAdd
            
            
            yrHistSt = yrHistEnd            
            tStepSt = tStepEnd
            
            if decade <7:                
                yrHistEnd = yrHistSt + 9
            else:
                yrHistEnd = yrHistSt + 5        
            
    else: 
        
    
        for contRun in range(1, 11):             
            
            tStepSt = 0
            memId = memId + 1
            yrCtrlDigits = str(yrCtrl)+'.' + str(contRun).rjust(3,'0')
            
            print(yrCtrlDigits)
            
            yrHistSt = 2014            
            yrHistEnd = yrHistSt + 9
            
            for decade in range(9):
                    
                yrHistSt = yrHistSt + 1
                yrHistEnd = yrHistEnd + 1
    
                yrMonHistSt = str(yrHistSt) + str(monSt).rjust(2,'0')
                yrMonHistEnd = str(yrHistEnd) + str(monEnd).rjust(2,'0')  
                
                
                fileName = 'b.e21.BSSP370cmip6.f09_g17.LE2-'+yrCtrlDigits+'.cam.h0.PRECT.' +yrMonHistSt+'-'+yrMonHistEnd+'.nc'
            
                ncFile_precip = str(filePath) + '/' + str(fileName)
                file_precip = Dataset(ncFile_precip, mode='r')                        
                param_precipToAdd = file_precip.variables['PRECT'][:, latli:latui , lonli:lonui]*86400*30*1000              
                t,y_US,x_US = param_precipToAdd.shape
                tStepEnd = tStepSt + t                  
                
                monPrecipUsScenEnsMem[tStepSt:tStepEnd, :, :, memId] = param_precipToAdd                 
            
                
                yrHistSt = yrHistEnd
                tStepSt = tStepEnd        
                
                
                if decade <7:                    
                    yrHistEnd = yrHistSt + 9
                else:
                    yrHistEnd = yrHistSt + 5      
    

1001.001
1021.002
1041.003
1061.004
1081.005
1101.006
1121.007
1141.008
1161.009
1181.010
1231.001
1231.002
1231.003
1231.004
1231.005
1231.006
1231.007
1231.008
1231.009
1231.010
1251.001
1251.002
1251.003
1251.004
1251.005
1251.006
1251.007
1251.008
1251.009
1251.010
1281.001
1281.002
1281.003
1281.004
1281.005
1281.006
1281.007
1281.008
1281.009
1281.010
1301.001
1301.002
1301.003
1301.004
1301.005
1301.006
1301.007
1301.008
1301.009
1301.010


In [24]:
print(monPrecipUsScenEnsMem.shape)

(1032, 32, 56, 50)


In [25]:
monPrecipUsScenEnsMean = np.nanmean(monPrecipUsScenEnsMem, axis=3)
print(monPrecipUsScenEnsMean.shape)

(1032, 32, 56)


In [26]:
np.save('/glade/scratch/mrhaider/cesm2/precip/monPrecipUsScenEnsMean.npy', monPrecipUsScenEnsMean)

In [30]:
STOP here ====================

SyntaxError: invalid syntax (<ipython-input-30-e2ad4fe443c1>, line 1)